In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crypto-regulations-yirifi/mappingtoBERT_6.csv
/kaggle/input/crypto-regulations-yirifi/mappingtoBERT_5.csv
/kaggle/input/crypto-regulations-yirifi/output_improved.csv
/kaggle/input/crypto-regulations-yirifi/master_data_gpt4o_improved_latest.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/18.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/20.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/1.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/24.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/11.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/30.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/17.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/16.csv
/kaggle/input/crypto-regulations-yirifi/CSV_data_gpt_4o/CSV_data_gpt_4o/3.csv
/kaggle/input/crypto-regulations-

# Data import and cleaning up the data

In [2]:
df = pd.read_csv("/kaggle/input/crypto-regulations-yirifi/mappingtoBERT_6.csv")

In [3]:
df.shape

(5965, 4)

In [4]:
df.tail(10)

,Unnamed: 0,Content,Concepts,bert_concepts
5955,5955,Content|ÄúRegulationsÄù|means the Virtual As...,"amended, related activities, 2023, supplemente...","regulations, virtual assets"
5956,5956,Content|ÄúReserve AssetsÄù|has the meaning a...,"frva rules, reserve assets, rule i.a.5",rules
5957,5957,Content|ÄúSenior ManagementÄù|has the meanin...,"company rulebook, senior management",rulebook
5958,5958,Content|ÄúSignificant FRVA IssuerÄù|means a ...,"rule i.c.1, vara, frva rules, vasp, significan...","rules, vasp"
5959,5959,Content|ÄúTechnology and Information\nRuleboo...,"amended, information, vara, technology, regula...","regulations, rulebook"
5960,5960,Content|ÄúVA Issuance RulebookÄù|means the V...,"amended, supplemented, vara, virtual asset iss...","regulations, virtual assets, rulebook"
5961,5961,Content|ÄúVARAÄù|means the Dubai Virtual Ass...,"vara, dubai, regulatory authority, virtual assets","authority, virtual assets, regulator"
5962,5962,Content|ÄúVirtual AssetÄù or ÄúVAÄù|has th...,"dubai va law, va, regulatory, compliance, virt...","compliance, virtual assets, regulator"
5963,5963,Content|ÄúWhitepaperÄù|has the meaning ascri...,"bank, whitepaper, regulatory compliance, compl...","compliance, virtual assets, rulebook, regulator"
5964,5964,Content|ÄúWorking DayÄù|means any day which ...,"public holiday, weekend, bank, hong kong, regu...","compliance, virtual assets, regulator"


In [5]:
import re
# Function to check for URLs
def contains_url(text):
    text = str(text)
    return bool(re.search(r'http\S+|www\S+|https\S+', text))

# Function to check for user references (@user)
def contains_user_reference(text):
    text = str(text)
    return bool(re.search(r'\@\w+', text))

# Function to check for hashtags
def contains_hashtag(text):
    text = str(text)
    return bool(re.search(r'\#\w+', text))

# Function to check for non-alphanumeric characters
def contains_non_alphanumeric(text):
    text = str(text)
    return bool(re.search(r'[^a-zA-Z0-9\s]', text))

# Apply the functions to the DataFrame
df['contains_url'] = df['Content'].apply(contains_url)
df['contains_user_reference'] = df['Content'].apply(contains_user_reference)
df['contains_hashtag'] = df['Content'].apply(contains_hashtag)
df['contains_non_alphanumeric'] = df['Content'].apply(contains_non_alphanumeric)

# Check rows that match each condition
url_count = df['contains_url'].sum()
user_reference_count = df['contains_user_reference'].sum()
hashtag_count = df['contains_hashtag'].sum()
non_alphanumeric_count = df['contains_non_alphanumeric'].sum()

# Print results
print(f"Number of rows with URLs: {url_count}")
print(f"Number of rows with user references: {user_reference_count}")
print(f"Number of rows with hashtags: {hashtag_count}")
print(f"Number of rows with non-alphanumeric characters: {non_alphanumeric_count}")



Number of rows with URLs: 22
Number of rows with user references: 12
Number of rows with hashtags: 0
Number of rows with non-alphanumeric characters: 5913


In [6]:
df.tail(5)

,Unnamed: 0,Content,Concepts,bert_concepts,contains_url,contains_user_reference,contains_hashtag,contains_non_alphanumeric
5960,5960,Content|ÄúVA Issuance RulebookÄù|means the V...,"amended, supplemented, vara, virtual asset iss...","regulations, virtual assets, rulebook",False,False,False,True
5961,5961,Content|ÄúVARAÄù|means the Dubai Virtual Ass...,"vara, dubai, regulatory authority, virtual assets","authority, virtual assets, regulator",False,False,False,True
5962,5962,Content|ÄúVirtual AssetÄù or ÄúVAÄù|has th...,"dubai va law, va, regulatory, compliance, virt...","compliance, virtual assets, regulator",False,False,False,True
5963,5963,Content|ÄúWhitepaperÄù|has the meaning ascri...,"bank, whitepaper, regulatory compliance, compl...","compliance, virtual assets, rulebook, regulator",False,False,False,True
5964,5964,Content|ÄúWorking DayÄù|means any day which ...,"public holiday, weekend, bank, hong kong, regu...","compliance, virtual assets, regulator",False,False,False,True


In [7]:
df = df[["Content","bert_concepts"]]

In [8]:
import re
import string

def preprocess_text(text):
    text = str(text)    
    text = re.sub(r'<[^>]+>', '', text) # Remove HTML tags
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)# Remove URLs
    text = re.sub(r'\@\w+|\#','', text) # Remove user @ references and hashtags
    text = text.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation    
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text) # Remove non-alphanumeric character    
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace(only single white space is left)
    
    return text

df['Content'] = df['Content'].apply(preprocess_text) # Applying the preprocess_text function


In [9]:
df.tail(5)

,Content,bert_concepts
5960,content va issuance rulebook means the virtual...,"regulations, virtual assets, rulebook"
5961,content vara means the dubai virtual assets re...,"authority, virtual assets, regulator"
5962,content virtual asset or va has the meaning as...,"compliance, virtual assets, regulator"
5963,content whitepaper has the meaning ascribed to...,"compliance, virtual assets, rulebook, regulator"
5964,content working day means any day which is not...,"compliance, virtual assets, regulator"


In [10]:
df.shape

(5965, 2)

In [11]:
missing_values_count = df['bert_concepts'].isnull().sum()
print("Number of rows with NaN in bert_concepts:", missing_values_count)

Number of rows with NaN in bert_concepts: 580


## Remove rows with no bert_concepts

In [12]:
df = df.dropna(subset=['bert_concepts'])

In [13]:
df.shape

(5385, 2)

# Converting to Bert Data Format

In [14]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [15]:
df.columns

Index(['Content', 'bert_concepts'], dtype='object')

In [16]:

df['bert_concepts'] = df['bert_concepts'].apply(lambda x: x.split(', ')) # make list of words
all_concepts = set(concept for row in df['bert_concepts'] for concept in row) # Create a list of all unique concepts


In [17]:
all_concepts

{'aml',
 'anti-bribery',
 'audit',
 'authority',
 'beneficiary information',
 'cdd',
 'cft',
 'circular',
 'client',
 'code of conduct',
 'compliance',
 'conduct',
 'controls',
 'criminal',
 'disclosure',
 'dlt',
 'due diligence',
 'entities',
 'guideline',
 'identity',
 'insider dealing',
 'investor',
 'investor protection',
 'kyc',
 'legal',
 'legislation',
 'license',
 'monitoring',
 'operator',
 'oversight',
 'pep',
 'person',
 'policies',
 'policy',
 'record keeping',
 'regulations',
 'regulator',
 'reporting',
 'requirements',
 'risk assessment',
 'risk management',
 'risks',
 'rulebook',
 'rules',
 'safeguard',
 'sanctions',
 'screening',
 'suspicious transactions',
 'transaction',
 'transactions',
 'transfer',
 'travel rule',
 'vasp',
 'vasps',
 'virtual assets'}

In [18]:
# One-hot encode each concept for each row
one_hot_encoded_rows = []
for index, row in df.iterrows():
    one_hot_row = {'Content': row['Content']}
    for concept in all_concepts:
        one_hot_row[concept] = True if concept in row['bert_concepts'] else False
    one_hot_encoded_rows.append(one_hot_row)


In [19]:
one_hot_encoded_rows[:2]

[{'Content': 'the new licensing regime for centralised virtual asset trading platforms under the antimoney laundering and counterterrorist financing ordinance cap 615 amlo will come into effect on 1 june 2023 under the new regime centralised virtual asset trading platforms operating in hong kong will need to apply to the securities and futures commission sfc for a licence under the securities and futures ordinance cap 571 sfo andor the amlo dual licence arrangement',
  'insider dealing': False,
  'authority': False,
  'legislation': False,
  'compliance': True,
  'beneficiary information': False,
  'due diligence': False,
  'travel rule': False,
  'kyc': False,
  'monitoring': False,
  'code of conduct': False,
  'cdd': False,
  'screening': False,
  'disclosure': False,
  'client': False,
  'virtual assets': True,
  'record keeping': False,
  'investor': False,
  'controls': False,
  'sanctions': False,
  'safeguard': False,
  'person': False,
  'risk assessment': False,
  'investor p

In [20]:
df = pd.DataFrame(one_hot_encoded_rows)

In [21]:
df.head(3)

,Content,insider dealing,authority,legislation,compliance,beneficiary information,due diligence,travel rule,kyc,monitoring,...,suspicious transactions,policy,policies,anti-bribery,oversight,entities,circular,conduct,operator,risk management
0,the new licensing regime for centralised virtu...,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,the sfc is issuing this circular to highlight ...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,the sfc has published the following guidelines...,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
import pandas as pd
from datasets import Dataset

In [23]:
#Splitting DataFrame into train, test, and validation sets
train_df, test_valid_df = train_test_split(df, test_size=0.3, random_state=42)
test_df, validation_df = train_test_split(test_valid_df, test_size=0.33, random_state=42)

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(validation_df)

# Create DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validation": validation_dataset
})




In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Content', 'insider dealing', 'authority', 'legislation', 'compliance', 'beneficiary information', 'due diligence', 'travel rule', 'kyc', 'monitoring', 'code of conduct', 'cdd', 'screening', 'disclosure', 'client', 'virtual assets', 'record keeping', 'investor', 'controls', 'sanctions', 'safeguard', 'person', 'risk assessment', 'investor protection', 'transfer', 'rulebook', 'guideline', 'dlt', 'risks', 'requirements', 'legal', 'criminal', 'transaction', 'rules', 'transactions', 'vasps', 'pep', 'cft', 'reporting', 'audit', 'regulations', 'identity', 'regulator', 'license', 'aml', 'vasp', 'suspicious transactions', 'policy', 'policies', 'anti-bribery', 'oversight', 'entities', 'circular', 'conduct', 'operator', 'risk management', '__index_level_0__'],
        num_rows: 3769
    })
    test: Dataset({
        features: ['Content', 'insider dealing', 'authority', 'legislation', 'compliance', 'beneficiary information', 'due diligence', '

In [25]:
[i for i in dataset['train']][:2]

[{'Content': 'a shall not by itself render that person liable to any judicial or other proceedings but in any proceedings under the sfo andor the amlo before any court these guidelines shall be admissible in evidence and if any provision set out in these guidelines appears to the court to be relevant to any question arising in the proceedings it shall be taken into account in determining the question and',
  'insider dealing': False,
  'authority': False,
  'legislation': False,
  'compliance': False,
  'beneficiary information': False,
  'due diligence': False,
  'travel rule': False,
  'kyc': False,
  'monitoring': False,
  'code of conduct': False,
  'cdd': False,
  'screening': False,
  'disclosure': False,
  'client': False,
  'virtual assets': False,
  'record keeping': False,
  'investor': False,
  'controls': False,
  'sanctions': False,
  'safeguard': False,
  'person': False,
  'risk assessment': False,
  'investor protection': False,
  'transfer': False,
  'rulebook': False,

# Data Ready Now Training

In [26]:
!pip install -q transformers datasets

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info/METADATA'



Add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.

### List containing all the labels

In [27]:
labels = [label for label in dataset['train'].features.keys() if label not in ['Content','__index_level_0__']]
#Create 2 dictionaries that map labels to integers and back.
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['insider dealing',
 'authority',
 'legislation',
 'compliance',
 'beneficiary information',
 'due diligence',
 'travel rule',
 'kyc',
 'monitoring',
 'code of conduct',
 'cdd',
 'screening',
 'disclosure',
 'client',
 'virtual assets',
 'record keeping',
 'investor',
 'controls',
 'sanctions',
 'safeguard',
 'person',
 'risk assessment',
 'investor protection',
 'transfer',
 'rulebook',
 'guideline',
 'dlt',
 'risks',
 'requirements',
 'legal',
 'criminal',
 'transaction',
 'rules',
 'transactions',
 'vasps',
 'pep',
 'cft',
 'reporting',
 'audit',
 'regulations',
 'identity',
 'regulator',
 'license',
 'aml',
 'vasp',
 'suspicious transactions',
 'policy',
 'policies',
 'anti-bribery',
 'oversight',
 'entities',
 'circular',
 'conduct',
 'operator',
 'risk management']

In [28]:
id2label

{0: 'insider dealing',
 1: 'authority',
 2: 'legislation',
 3: 'compliance',
 4: 'beneficiary information',
 5: 'due diligence',
 6: 'travel rule',
 7: 'kyc',
 8: 'monitoring',
 9: 'code of conduct',
 10: 'cdd',
 11: 'screening',
 12: 'disclosure',
 13: 'client',
 14: 'virtual assets',
 15: 'record keeping',
 16: 'investor',
 17: 'controls',
 18: 'sanctions',
 19: 'safeguard',
 20: 'person',
 21: 'risk assessment',
 22: 'investor protection',
 23: 'transfer',
 24: 'rulebook',
 25: 'guideline',
 26: 'dlt',
 27: 'risks',
 28: 'requirements',
 29: 'legal',
 30: 'criminal',
 31: 'transaction',
 32: 'rules',
 33: 'transactions',
 34: 'vasps',
 35: 'pep',
 36: 'cft',
 37: 'reporting',
 38: 'audit',
 39: 'regulations',
 40: 'identity',
 41: 'regulator',
 42: 'license',
 43: 'aml',
 44: 'vasp',
 45: 'suspicious transactions',
 46: 'policy',
 47: 'policies',
 48: 'anti-bribery',
 49: 'oversight',
 50: 'entities',
 51: 'circular',
 52: 'conduct',
 53: 'operator',
 54: 'risk management'}

# Tokenization

### As models like BERT don't expect text as direct input, but rather input_ids, etc., we tokenize the text using the tokenizer. We will use AutoTokenizer API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

### What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' BCEWithLogitsLoss (which the model will use) will complain

In [29]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

def preprocess_data(examples, max_length=128):
  # take a batch of texts
  text = examples['Content']
  batch_size = len(text)
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=max_length)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [30]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/3769 [00:00<?, ? examples/s]

Map:   0%|          | 0/1082 [00:00<?, ? examples/s]

Map:   0%|          | 0/534 [00:00<?, ? examples/s]

In [31]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3769
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1082
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 534
    })
})

In [32]:
encoded_dataset['train']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3769
})

In [33]:
example = encoded_dataset['train'][0]
tokenizer.decode(example['input_ids'])

2024-06-05 05:04:25.208076: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 05:04:25.208174: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 05:04:25.335004: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'[CLS] a shall not by itself render that person liable to any judicial or other proceedings but in any proceedings under the sfo andor the amlo before any court these guidelines shall be admissible in evidence and if any provision set out in these guidelines appears to the court to be relevant to any question arising in the proceedings it shall be taken into account in determining the question and [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [34]:
#example['labels']

In [35]:
#use id to label dictionary
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]
encoded_dataset.set_format("torch") 

### Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

### We set the problem_type to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely BCEWithLogitsLoss). We also make sure the output layer has len(labels) output neurons, and we set the id2label and label2id mappings

In [36]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id,
                                                          ignore_mismatched_sizes=True )

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([55, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([55]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
batch_size = 16
metric_name = "f1"

In [38]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


we need to define a compute_metrics function, that returns a dictionary with the desired metric values.

source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

In [39]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds,labels=p.label_ids)
    return result
  
   


In [40]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [41]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1037,  4618,  2025,  2011,  2993, 17552,  2008,  2711, 20090,
         2000,  2151,  8268,  2030,  2060,  8931,  2021,  1999,  2151,  8931,
         2104,  1996, 16420,  2080,  1998,  2953,  1996,  2572,  4135,  2077,
         2151,  2457,  2122, 11594,  4618,  2022,  4748, 26770,  1999,  3350,
         1998,  2065,  2151,  9347,  2275,  2041,  1999,  2122, 11594,  3544,
         2000,  1996,  2457,  2000,  2022,  7882,  2000,  2151,  3160, 17707,
         1999,  1996,  8931,  2009,  4618,  2022,  2579,  2046,  4070,  1999,
        12515,  1996,  3160,  1998,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [42]:
# #forward pass
# #, attention_mask=attention_mask
# outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
# outputs

In [43]:
from transformers import AutoModelForSequenceClassification, Trainer

In [44]:

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")


In [45]:
import os
os.environ['WANDB_API_KEY'] ="313a57558bcaee784e68d1654f7915a0b463a341"

In [46]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

In [47]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: anshtanwar007 (anshta). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240605_050443-zj09zluc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bert-finetuned-sem_eval-english
wandb: ⭐️ View project at https://wandb.ai/anshta/huggingface
wandb: 🚀 View run at https://wandb.ai/anshta/huggingface/runs/zj09zluc


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.175252,0.131980,0.535371,0.005618
2,No log,0.138936,0.529551,0.684789,0.144195
3,0.191400,0.119717,0.604634,0.725708,0.192884
4,0.191400,0.108046,0.687413,0.770596,0.286517
5,0.109100,0.100622,0.726542,0.798110,0.307116
6,0.109100,0.094908,0.737787,0.805318,0.329588
7,0.081500,0.090828,0.759948,0.820492,0.357678
8,0.081500,0.087802,0.772436,0.831300,0.382022
9,0.064500,0.086821,0.773537,0.834106,0.378277
10,0.064500,0.086254,0.778592,0.835965,0.385768


TrainOutput(global_step=2360, training_loss=0.1036863755371611, metrics={'train_runtime': 564.0313, 'train_samples_per_second': 66.823, 'train_steps_per_second': 4.184, 'total_flos': 2480343667991040.0, 'train_loss': 0.1036863755371611, 'epoch': 10.0})

In [48]:
trainer.evaluate()

{'eval_loss': 0.08625388145446777,
 'eval_f1': 0.7785919082510354,
 'eval_roc_auc': 0.8359651177425891,
 'eval_accuracy': 0.3857677902621723,
 'eval_runtime': 2.0966,
 'eval_samples_per_second': 254.698,
 'eval_steps_per_second': 16.217,
 'epoch': 10.0}

# Inference
add Codeadd Markdown
The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1.

The logits is a tensor that contains the (unnormalized) scores for every individual label.

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1.

The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [49]:
def get_answer(text):
    
    encoding = tokenizer(text, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
    outputs = trainer.model(**encoding)
    logits = outputs.logits
    print(logits.shape)
    
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    print(probs)
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= 0.5)] = 1
    print(predictions)
    predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
    print()
    return predicted_labels


In [50]:
text = "The new licensing regime for centralised virtual asset trading platforms under the Anti-MoneyLaundering and Counter-Terrorist Financing Ordinance (Cap. 615) (AMLO) will come intoeffect on 1 June 2023. Under the new regime, centralised virtual asset trading platformsoperating in Hong Kong will need to apply to the Securities and Futures Commission (SFC)for a licence under the Securities and Futures Ordinance (Cap 571) (SFO) and/or the AMLO(Dual Licence Arrangement)"
get_answer(text)

torch.Size([1, 55])
tensor([0.0073, 0.0653, 0.0307, 0.0958, 0.0135, 0.0121, 0.0095, 0.0055, 0.0128,
        0.0052, 0.0245, 0.0105, 0.0086, 0.0075, 0.7651, 0.0106, 0.0090, 0.0260,
        0.0101, 0.0047, 0.0718, 0.0195, 0.0128, 0.0373, 0.0073, 0.1283, 0.0123,
        0.0436, 0.0329, 0.0324, 0.0242, 0.0337, 0.0179, 0.0249, 0.0215, 0.0116,
        0.7817, 0.0196, 0.0181, 0.0461, 0.0103, 0.1734, 0.8932, 0.9225, 0.0326,
        0.0124, 0.0103, 0.0146, 0.0032, 0.0135, 0.0254, 0.0146, 0.0169, 0.0493,
        0.0111], grad_fn=<SigmoidBackward0>)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]



['virtual assets', 'cft', 'license', 'aml']

# Probability

In [51]:
# outputs = trainer.model(**encoding)
# logits = outputs.logits
# logits.shape

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1.

The logits is a tensor that contains the (unnormalized) scores for every individual label

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example)

In [52]:
# # apply sigmoid + threshold
# sigmoid = torch.nn.Sigmoid()
# probs = sigmoid(logits.squeeze().cpu())
# predictions = np.zeros(probs.shape)
# predictions[np.where(probs >= 0.5)] = 1
# # turn predicted id's into actual label names
# predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
# print(predicted_labels)